# Test speed of loading different filetypes

In [19]:
import os
from timeit import default_timer

import numpy as np
import h5py

import plotly.express as px

In [20]:
data_path = os.path.normpath("../../data")
folders = [os.path.join(data_path, "gen_plasma_n64", f"mat_{i}") for i in range(3)]
lens = [int(len(list(filter(lambda x: x.endswith(".csv"), os.listdir(f)))) / 4) for f in folders]

print(folders, lens)

['../../data/gen_plasma_n64/mat_0', '../../data/gen_plasma_n64/mat_1', '../../data/gen_plasma_n64/mat_2'] [8145, 8145, 5685]


## CSV format

### Loading one matrix

In [22]:
times_csv = []

for i, folder in enumerate(folders):
    time = 0

    for f in range(lens[i]):
        path = os.path.join(folder, f"f_{f}.csv")

        start_time = default_timer()
        tstload = np.genfromtxt(path, delimiter=",", dtype=float)
        time += default_timer() - start_time

    times_csv.append(time / lens[i])
    print(times_csv)

KeyboardInterrupt: 

## Convert to HDF5

Only run this code once.

### Loading all 4 matrices

In [23]:
# paths = [os.path.join(folders[0], f"{s}_0.csv") for s in ("f", "S", "U", "V")]

# m_f = np.genfromtxt(paths[0], delimiter=",", dtype=float)
# m_S = np.genfromtxt(paths[1], delimiter=",", dtype=float)
# m_U = np.genfromtxt(paths[2], delimiter=",", dtype=float)
# m_V = np.genfromtxt(paths[3], delimiter=",", dtype=float)

# with h5py.File(os.path.join(folders[0], '0.hdf5'), 'w') as file:
#     file.create_dataset("f", data=m_f)
#     file.create_dataset("S", data=m_S)
#     file.create_dataset("U", data=m_U)
#     file.create_dataset("V", data=m_V)

In [30]:
# with h5py.File(os.path.join(folders[0], '0.hdf5'), 'r') as file:
#     print(file, file.keys())
#     print(file["f"])
#     new_f = file["f"][()] # The syntax actually retrtrieves and stores the data

# print(new_f, new_f.shape, new_f.size, new_f.dtype, type(new_f), new_f[0, 30])


<HDF5 file "0.hdf5" (mode r)> <KeysViewHDF5 ['S', 'U', 'V', 'f']>
<HDF5 dataset "f": shape (64, 128), type "<f8">
[[2.15367463e-09 3.93651655e-09 7.16342953e-09 ... 7.31011585e-09
  4.02170458e-09 2.18977458e-09]
 [2.12611971e-09 3.87617972e-09 7.05779149e-09 ... 7.34854840e-09
  4.04503671e-09 2.19767585e-09]
 [2.09272750e-09 3.80517765e-09 6.93264068e-09 ... 7.36230817e-09
  4.05470692e-09 2.19846995e-09]
 ...
 [2.19767584e-09 4.04503670e-09 7.34854840e-09 ... 7.05779149e-09
  3.87617972e-09 2.12611972e-09]
 [2.18977457e-09 4.02170458e-09 7.31011585e-09 ... 7.16342953e-09
  3.93651655e-09 2.15367464e-09]
 [2.17500809e-09 3.98529515e-09 7.24800450e-09 ... 7.24800450e-09
  3.98529515e-09 2.17500809e-09]] (64, 128) 8192 float64 <class 'numpy.ndarray'> 0.00265982621971962


In [ ]:
times_allm_csv = []

for i, folder in enumerate(folders):
    time = 0
    
    for f in range(lens[i]):
        paths = [os.path.join(folder, f"{s}_{f}.csv") for s in (   i n   [ " f " ,)]

        start_time = default_timer()
        m_f = np.genfromtxt(paths[0], delimiter=",", dtype=float)
        m_S = np.genfromtxt(paths[1], delimiter=",", dtype=float)
        m_U = np.genfromtxt(paths[2], delimiter=",", dtype=float)
        m_V = np.genfromtxt(paths[3], delimiter=",", dtype=float)
        time += default_timer() - start_time

        with h5py.File(os.path.join(folder, f'{f}.hdf5'), 'w') as file:
                file.create_dataset("f", data=m_f)
                file.create_dataset("S", data=m_S)
                file.create_dataset("U", data=m_U)
                file.create_dataset("V", data=m_V)

    times_allm_csv.append(time / lens[i])
    print(f"Done {folder}, times: {times_allm_csv}")


## HDF5 Format

## MAT Format

Assumes that the mats were generated by Matlab; cannot use the original plasma gen code, need to actually load the csvs and save as `.mat`s becuse need to be the exact same data.